In [2]:
pip install opencv-python pandas

    pytz (>dev)
         ~^
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import cv2
import pandas as pd
import tempfile

def merge_videos_from_folder(video_folder, excel_path, sheet_name, output_dir):
    """
    Функция для обработки видео из папки и объединения их согласно данным из Excel.
    
    :param video_folder: Путь к папке с видеофайлами
    :param excel_path: Путь к файлу Excel с данными
    :param sheet_name: Название листа в Excel
    :param output_dir: Директория для сохранения результатов
    """
    # Создаем все необходимые директории
    os.makedirs(output_dir, exist_ok=True)
    
    # Создаем временную директорию для работы с видео
    temp_dir = tempfile.mkdtemp()
    
    try:
        # Читаем данные из указанного листа Excel
        df = pd.read_excel(excel_path, sheet_name=sheet_name)
        
        # Проверяем наличие необходимых колонок
        required_columns = ['#', 'Игрок', 'Видео ID']
        if not all(col in df.columns for col in required_columns):
            raise ValueError("В файле Excel отсутствуют необходимые колонки!")

        # Получаем список всех видеофайлов в папке (без учета регистра)
        video_files_in_folder = {os.path.basename(f).lower(): os.path.join(video_folder, f) 
                               for f in os.listdir(video_folder) 
                               if f.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))}
            
        # Обрабатываем каждую строку в Excel
        for _, row in df.iterrows():
            player_number = str(row['#']).strip()
            player_name = str(row['Игрок']).strip()
            video_ids = [vid.strip() for vid in str(row['Видео ID']).split()]
            
            valid_clips = []
            
            # Собираем все видеофрагменты для текущего игрока
            for vid_id in video_ids:
                found = False
                # Проверяем возможные расширения файлов
                for ext in ['.mp4', '.mov', '.avi', '.mkv']:
                    expected_filename = f"player_{vid_id}{ext}".lower()
                    
                    if expected_filename in video_files_in_folder:
                        video_path = video_files_in_folder[expected_filename]
                        try:
                            # Проверяем, можно ли открыть видео
                            cap = cv2.VideoCapture(video_path)
                            if cap.isOpened():
                                valid_clips.append(video_path)
                                found = True
                                cap.release()
                            else:
                                print(f"Ошибка: не удалось открыть видео {vid_id}{ext}")
                        except Exception as e:
                            print(f"Ошибка при обработке видео {vid_id}: {str(e)}")
                        break
                
                if not found:
                    print(f"Видеофрагмент {vid_id} не найден в папке")
            
            # Если есть видео для объединения
            if valid_clips:
                output_filename = f"{player_number}_{player_name}_shifts.mp4"
                output_path = os.path.join(output_dir, output_filename)
                
                try:
                    # Получаем параметры видео из первого клипа
                    first_clip = cv2.VideoCapture(valid_clips[0])
                    fps = first_clip.get(cv2.CAP_PROP_FPS)
                    width = int(first_clip.get(cv2.CAP_PROP_FRAME_WIDTH))
                    height = int(first_clip.get(cv2.CAP_PROP_FRAME_HEIGHT))
                    first_clip.release()
                    
                    # Создаем VideoWriter
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
                    
                    # Объединяем видео
                    for clip_path in valid_clips:
                        cap = cv2.VideoCapture(clip_path)
                        while True:
                            ret, frame = cap.read()
                            if not ret:
                                break
                            out.write(frame)
                        cap.release()
                    
                    out.release()
                    print(f"Успешно создан файл: {output_filename}")
                except Exception as e:
                    print(f"Ошибка при объединении видео для {player_name}: {str(e)}")
            else:
                print(f"Нет доступных видеофрагментов для {player_name}")
    
    finally:
        # Удаляем временную директорию
        try:
            for root, dirs, files in os.walk(temp_dir, topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
            os.rmdir(temp_dir)
        except:
            pass

In [5]:
video_folder = "/path/to/generated/player_videos"
excel_path = "/path/to/your/Analytics.xlsx"
sheet_name = "Stats"
output_dir = "/path/to/save/output/processed_videos"
    
merge_videos_from_folder(video_folder, excel_path, sheet_name, output_dir)

Успешно создан файл: 7_Dzhioshvili_shifts.mp4
Успешно создан файл: 10_Comtois_shifts.mp4
Успешно создан файл: 12_Chernov_shifts.mp4
Успешно создан файл: 13_Mikheyev_shifts.mp4
Успешно создан файл: 18_Paquette_shifts.mp4
Успешно создан файл: 20_Buruyanov_shifts.mp4
Успешно создан файл: 23_Kudashov_shifts.mp4
Успешно создан файл: 27_Menell_shifts.mp4
Видеофрагмент nan не найден в папке
Нет доступных видеофрагментов для Gotovets
Успешно создан файл: 44_Adamchuk_shifts.mp4
Успешно создан файл: 45_Pylenkov_shifts.mp4
Видеофрагмент nan не найден в папке
Нет доступных видеофрагментов для Burmistrov
Успешно создан файл: 84_Kudryavtsev_shifts.mp4
Видеофрагмент nan не найден в папке
Нет доступных видеофрагментов для Rimashevsky
Успешно создан файл: 90_Sikura_shifts.mp4
Успешно создан файл: 91_Weal_shifts.mp4
Успешно создан файл: 92_Ozhiganov_shifts.mp4
Успешно создан файл: 93_Sergeyev_shifts.mp4
Успешно создан файл: 96_Rashevsky_shifts.mp4
Успешно создан файл: 97_Gusev_shifts.mp4
Успешно создан 